FETCH HISTORIC DATA FROM YFINANCE

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import date

start_date = "2020-01-01"
end_date = "2025-03-15"


df = pd.read_csv("../datasets/tickers.csv")
symbols = df["symbol"].tolist()

data = pd.DataFrame()

for symbol in symbols:
    try:
        ticker_data = yf.download(symbol, start=start_date, end=end_date, auto_adjust=True)
        
        ticker_data.columns = ticker_data.columns.get_level_values(0)
        ticker_data = ticker_data.reset_index()
        ticker_data = ticker_data.rename(columns={"price": "id"})
        ticker_data['symbol'] = symbol
        
        data = pd.concat([data, ticker_data], ignore_index=True)
    except Exception as e:
            print(f"Fetch error {symbol}: {e}")

print(data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Price        Date       Close        High         Low        Open     Volume  \
0      2020-01-02   20.655466   21.079762   20.554214   21.079762  3062500.0   
1      2020-01-03   20.732611   20.838685   20.395103   20.414389  3011800.0   
2      2020-01-06   20.250460   20.684399   20.154029   20.578324  3060400.0   
3      2020-01-07   20.559040   20.776010   20.240819   20.308320  4986900.0   
4      2020-01-08   19.729734   20.626541   19.671876   20.539753  3706300.0   
...           ...         ...         ...         ...         ...        ...   
647634 2025-03-10  194.690002  204.600006  192.740005  203.470001  3057300.0   
647635 2025-03-11  199.360001  204.414993  195.199997  196.404999  2431700.0   
647636 2025-03-12  197.789993  203.970001  195.160004  202.940002  2217000.0   
647637 2025-03-13  189.580002  196.494995  186.669998  196.179993  2491900.0   
647638 2025-03-14  197.809998  198.000000  192.000000  192.770004  1581000.0   

Price  symbol  Adj Close  
0          A

In [2]:
data.to_csv('../datasets/yahoo_historic_data.csv',index=False)

HISTORIC DATASET REVIEW

In [10]:
import pandas as pd

df = pd.read_csv('../datasets/yahoo_historic_data.csv')
df

,Date,Close,High,Low,Open,Volume,symbol,Adj Close
0,2020-01-02,20.655466,21.079762,20.554214,21.079762,3062500.0,AA,NaN
1,2020-01-03,20.732611,20.838685,20.395103,20.414389,3011800.0,AA,NaN
2,2020-01-06,20.250460,20.684399,20.154029,20.578324,3060400.0,AA,NaN
3,2020-01-07,20.559040,20.776010,20.240819,20.308320,4986900.0,AA,NaN
4,2020-01-08,19.729734,20.626541,19.671876,20.539753,3706300.0,AA,NaN
...,...,...,...,...,...,...,...,...
647634,2025-03-10,194.690002,204.600006,192.740005,203.470001,3057300.0,ZS,NaN
647635,2025-03-11,199.360001,204.414993,195.199997,196.404999,2431700.0,ZS,NaN
647636,2025-03-12,197.789993,203.970001,195.160004,202.940002,2217000.0,ZS,NaN
647637,2025-03-13,189.580002,196.494995,186.669998,196.179993,2491900.0,ZS,NaN


In [5]:
#remove stocks that have less than 1000 days of history
pocet_radku_alt = df["symbol"].value_counts()

symbols_to_keep = pocet_radku_alt[pocet_radku_alt >= 1000].index
df = df[df["symbol"].isin(symbols_to_keep)]

In [6]:
unique_symbols = df["symbol"].nunique()
print("Počet unikátních symbolů:", unique_symbols)

rows_on_date = df[df["Date"] == "2025-03-14"].shape[0]
print("Počet řádků s datem 2025-03-14:", rows_on_date)

#rows with empty values
print(df.isnull().sum())


Počet unikátních symbolů: 477
Počet řádků s datem 2025-03-14: 477
Date              0
Close             0
High              0
Low               0
Open              0
Volume            0
symbol            0
Adj Close    614932
dtype: int64


In [7]:
df.columns = df.columns.str.lower()
df

,date,close,high,low,open,volume,symbol,adj close
0,2020-01-02,20.655466,21.079762,20.554214,21.079762,3062500.0,AA,NaN
1,2020-01-03,20.732611,20.838685,20.395103,20.414389,3011800.0,AA,NaN
2,2020-01-06,20.250460,20.684399,20.154029,20.578324,3060400.0,AA,NaN
3,2020-01-07,20.559040,20.776010,20.240819,20.308320,4986900.0,AA,NaN
4,2020-01-08,19.729734,20.626541,19.671876,20.539753,3706300.0,AA,NaN
...,...,...,...,...,...,...,...,...
647634,2025-03-10,194.690002,204.600006,192.740005,203.470001,3057300.0,ZS,NaN
647635,2025-03-11,199.360001,204.414993,195.199997,196.404999,2431700.0,ZS,NaN
647636,2025-03-12,197.789993,203.970001,195.160004,202.940002,2217000.0,ZS,NaN
647637,2025-03-13,189.580002,196.494995,186.669998,196.179993,2491900.0,ZS,NaN


In [8]:

rearrange_columns = ['date','symbol','open','high','low','close','volume']

df = df[rearrange_columns]
df.to_csv('../datasets/hist_data.csv', index=False)

In [9]:
#stocks that have at least 1000 days of history

symbols = df['symbol'].unique()

df_symbols = pd.DataFrame(symbols, columns=['symbol'])

df_symbols.to_csv('../datasets/new_tickers_list.csv', index=False)

FETCH COMPANY INFORMATION DATA FROM YFINANCE

In [3]:
import pandas as pd
import yfinance as yf
from datetime import date

start_date = "2020-01-01"
end_date = date.today().strftime("%Y-%m-%d")  # ve formátu YYYY-MM-DD

# Načtení seznamu tickerů ze souboru CSV
df = pd.read_csv("../datasets/new_tickers_list.csv")
symbols = df["symbol"].tolist()

data = pd.DataFrame()

for symbol in symbols:
    try:
        stock = yf.Ticker(symbol)
        info = stock.info

        company_data = {
                "symbol": symbol,
                "short_name": info.get("shortName"),
                "region": info.get("region"),
                "exchange": info.get("exchange"),
                "exchange_timezone": info.get("exchangeTimezoneShortName"),
                "market_cap": info.get("marketCap"),
                "beta": info.get("beta"),
        }
        new_df = pd.DataFrame([company_data])
        
        data = pd.concat([data, new_df], ignore_index=True)
    except Exception as e:
            print(f"Fetch error {symbol}: {e}")

print(data)

C:\Users\komis\AppData\Local\Temp\ipykernel_4472\512802057.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, new_df], ignore_index=True)


    symbol                       short_name region exchange exchange_timezone  \
0       AA                Alcoa Corporation     US      NYQ               EDT   
1      AAL    American Airlines Group, Inc.     US      NMS               EDT   
2      AAP          Advance Auto Parts Inc.     US      NYQ               EDT   
3     AAPL                       Apple Inc.     US      NMS               EDT   
4     ABBV                      AbbVie Inc.     US      NYQ               EDT   
..     ...                              ...    ...      ...               ...   
473      Z               Zillow Group, Inc.     US      NMS               EDT   
474     ZI       ZoomInfo Technologies Inc.     US      NMS               EDT   
475    ZIM  ZIM Integrated Shipping Service     US      NYQ               EDT   
476     ZM        Zoom Communications, Inc.     US      NMS               EDT   
477     ZS                    Zscaler, Inc.     US      NMS               EDT   

        market_cap   beta  

In [ ]:
data.to_csv('../datasets/comp_data.csv',index=False)

COMPANY INFO DATASET REVIEW

In [ ]:
import pandas as pd

df = pd.read_csv('../datasets/comp_data.csv')

In [2]:
df.head()

,symbol,short_name,region,exchange,exchange_timezone,market_cap,beta
0,AA,Alcoa Corporation,US,NYQ,EDT,8680380416,2.553
1,AAL,"American Airlines Group, Inc.",US,NMS,EDT,7147851264,1.270
2,AAP,Advance Auto Parts Inc.,US,NYQ,EDT,2247615232,1.293
3,AAPL,Apple Inc.,US,NMS,EDT,3207068385280,1.178
4,ABBV,AbbVie Inc.,US,NYQ,EDT,373848178688,0.598


In [3]:
print(df.isnull().sum())

symbol               0
short_name           0
region               0
exchange             0
exchange_timezone    0
market_cap           0
beta                 2
dtype: int64


SEEED DATA INTO DATABASE

In [1]:
import os
import psycopg2
from dotenv import load_dotenv

load_dotenv()


DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)


cursor = conn.cursor()

print("✅ Connect to database!")


✅ Connect to database!


In [2]:
import pandas as pd

df_company = pd.read_csv("../datasets/comp_data.csv")
df_history = pd.read_csv("../datasets/hist_data.csv")

In [ ]:
import numpy as np

sql_companies = """
    INSERT INTO companies (symbol, short_name, region, exchange, exchange_timezone,
                        market_cap, beta )
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

sql_historic_data = """
    INSERT INTO daily_historic_data (date, symbol, open, high, low, close,
                        volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""



def prepare_values_for_sql(df):
    df = df.astype(object)
    
    df = df.where(pd.notna(df), None)
    
    values = [
        tuple(x.item() if isinstance(x, (np.integer, np.floating)) else x for x in row)
        for row in df.to_records(index=False)
    ]
    
    return values

cursor.executemany(sql_companies, prepare_values_for_sql(df_company))
cursor.executemany(sql_historic_data, prepare_values_for_sql(df_history))

conn.commit()

In [4]:
cursor.close()
conn.close()